# Who will win the 2020 Masters?

Analysis is paused til next quarter.  Will revisit then!


## Data set
I chose PGA Tour ShotLink data from 1980 to 2019 (www.shotlink.com and www.pgatour.com).  This is the largest set of data available on these metrics.  The data is manually collected by volunteers who follow every PGA Tour player and record every shot made in every PGA Tour tournament.  I was looking into scraping the data when I realized someone on Kaggle had already done this (https://www.kaggle.com/bradklassen/pga-tour-20102018-data).  I instead used this data set, formatted it, and cleaned out obvious errors.


## Conclusion
TBD!


# Detailed steps shown below

## Importing, Formatting and Cleaning the Data

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from scipy import stats
sns.set_palette("GnBu_d")

In [2]:
df = pd.read_csv('pga_data_historical.csv') #2010-2019 PGA Tour data, source: https://www.kaggle.com/bradklassen/pga-tour-20102018-data (as of August 2020, he has taken the data down)

In [3]:
df.head()

,player_name,date,tournament,statistic,variable,value
0,Rik Massengale,1980-01-13,Bob Hope Desert Classic,Final Round Scoring Average,AVG,70.00
1,Bobby Nichols,1980-01-13,Bob Hope Desert Classic,Final Round Scoring Average,AVG,73.00
2,Andy North,1980-01-13,Bob Hope Desert Classic,Final Round Scoring Average,AVG,73.00
3,John Mahaffey,1980-01-13,Bob Hope Desert Classic,Final Round Scoring Average,AVG,73.00
4,Peter Jacobsen,1980-01-13,Bob Hope Desert Classic,Final Round Scoring Average,AVG,73.00


In [4]:
df.shape

(46147897, 6)

In [5]:
#Unearthing the available variables in the "statistic" column
df.statistic.unique()

array(['Final Round Scoring Average', 'All-Around Ranking',
       'Par 4 Birdie or Better Leaders', 'Scoring Average Before Cut',
       'Scoring Average (Actual)', 'Round 3 Scoring Average',
       'Official Money', 'Greens in Regulation Percentage',
       'Total Birdies', 'Birdie or Better Conversion Percentage',
       'Eagles (Holes per)', 'Total Driving', 'Scoring Average',
       'Par 3 Birdie or Better Leaders', 'Driving Distance',
       'Sand Save Percentage', 'Putts Per Round',
       'Percentage of potential money won', 'Driving Accuracy Percentage',
       'Par 5 Birdie or Better Leaders', 'Par Breakers', 'Ball Striking',
       'Birdie Average', 'Total Money (Official and Unofficial)',
       'Total Eagles', 'Front 9 Round 1 Scoring Average',
       'Back 9 Round 1 Scoring Average', 'Back 9 Par 5 Scoring Average',
       'Front 9 Round 2 Scoring Average', 'Front 9 Par 4 Scoring Average',
       'Back 9 Par 3 Scoring Average', 'Front 9 Par 3 Scoring Average',
       'Fron

In [6]:
df.statistic.value_counts()

Total Eagles                               256346
Total Money (Official and Unofficial)      256156
Total Birdies                              255870
Percentage of potential money won          254714
Final Round Scoring Average                252576
                                            ...  
Late Round 5 Scoring Average                  890
Tenth Tee Early Round 5 Scoring Average       528
First Tee Late Round 5 Scoring Average        468
Tenth Tee Late Round 5 Scoring Average        422
First Tee Early Round 5 Scoring Average       276
Name: statistic, Length: 442, dtype: int64

In [7]:
#Selecting from the hundreds of variables the "statistic" column

chosen_statistic = ['Greens in Regulation Percentage',
                    'Driving Accuracy Percentage',
                    'Scrambling',
                    'Scoring Average (Actual)',
                    'Official Money',
                    'Percentage of Available Purse Won',
                    'Driving Distance',
                    'Sand Save Percentage',
                    'Overall Putting Average']

#Definitions for each variable:
    #Greens in Regulation Percentage: The percent of time a player was able to hit the green in regulation (greens hit in regulation/holes played). Note: A green is considered hit in regulation if any portion of the ball is touching the putting surface after the GIR stroke has been taken. (The GIR stroke is determined by subtracting 2 from par (1st stroke on a par 3, 2nd on a par 4, 3rd on a par 5))
    #Driving Accuracy Percentage: The percentage of time a tee shot comes to rest in the fairway (regardless of club)
    #Scrambling: The percent of time a player misses the green in regulation, but still makes par or better.
    #Scoring Average (Actual): The average number of strokes per completed round.
    #Official Money: The total official money a player has earned year-to-date. Note: This is for PGA TOUR members only.
    #Percentage of Available Purse Won: For official events, the player's total money won as a percentage of the total purse available.
    #Driving Distance: The average number of yards per measured drive. These drives are measured on two holes per round. Care is taken to select two holes which face in opposite directions to counteract the effect of wind. Drives are measured to the point at which they come to rest regardless of whether they are in the fairway or not.
    #Sand Save Percentage: The percent of time a player was able to get 'up and down' once in a greenside sand bunker (regardless of score). Note: 'Up and down' indicates it took the player 2 shots or less to put the ball in the hole from that point.
    #Overall Putting Average: The average number of putts for all holes played (total putts / total holes played).

In [8]:
#Adjusting the DataFrame
df = df[df['statistic'].isin(chosen_statistic)]
df = df[df['variable'] != 'RANK THIS WEEK'] #Multiple variables have the value 'RANK THIS WEEK', so unstacking will create duplicate columns of 'RANK THIS WEEK', causing an error ("Index contains duplicate entries, cannot reshape"). This line eliminates the error
df = df.drop(columns = ['variable'])
df.head()

,player_name,date,tournament,statistic,value
145,Bill Rogers,1980-01-13,Bob Hope Desert Classic,Scoring Average (Actual),70.60
146,Gil Morgan,1980-01-13,Bob Hope Desert Classic,Scoring Average (Actual),70.60
147,Bob Gilder,1980-01-13,Bob Hope Desert Classic,Scoring Average (Actual),70.60
148,Billy Kratzert,1980-01-13,Bob Hope Desert Classic,Scoring Average (Actual),70.60
149,Roger Maltbie,1980-01-13,Bob Hope Desert Classic,Scoring Average (Actual),70.60


In [9]:
#Unstacking the data in the DataFrame
df = df.set_index(['player_name', 'date', 'tournament','statistic'])['value'].unstack('statistic').reset_index()
df.head()

statistic,player_name,date,tournament,Driving Accuracy Percentage,Driving Distance,Greens in Regulation Percentage,Official Money,Overall Putting Average,Percentage of Available Purse Won,Sand Save Percentage,Scoring Average (Actual),Scrambling
0,A.J. Duncan,1988-04-03,KMart Greater Greensboro Open,58.93,283.5,58.33,"$1,810",NaN,.18,33.33,73.75,NaN
1,A.J. Duncan,1990-03-11,Honda Classic,44.64,264.1,52.78,"$1,810",NaN,.18,14.29,77.00,NaN
2,A.J. McInerney,2017-11-05,Shriners Hospitals for Children Open,50.00,317.8,76.39,NaN,1.694,2.22,22.22,69.50,52.94
3,A.J. McInerney,2018-06-10,FedEx St. Jude Classic,50.00,298.5,56.94,NaN,1.514,.22,33.33,70.75,61.29
4,Aaron Baddeley,2000-03-12,Honda Classic,64.29,283.6,70.83,NaN,1.625,NaN,50.00,70.25,66.67


In [10]:
df.shape

(128160, 12)

In [11]:
df.describe()

statistic,player_name,date,tournament,Driving Accuracy Percentage,Driving Distance,Greens in Regulation Percentage,Official Money,Overall Putting Average,Percentage of Available Purse Won,Sand Save Percentage,Scoring Average (Actual),Scrambling
count,128160,128160,128160,124669,123306,124543,123674,87568,118148,123011,126125,87574
unique,2440,1697,304,333,1169,111,11277,206,772,134,129,425
top,Davis Love III,1995-07-23,PGA Championship,64.29,271.9,66.67,$,1.611,.22,50.00,70.00,66.67
freq,546,178,3018,7030,322,8056,200,6397,7025,19184,6630,4253


## Fixing the Data types

In [12]:
print(df.dtypes)

statistic
player_name                          object
date                                 object
tournament                           object
Driving Accuracy Percentage          object
Driving Distance                     object
Greens in Regulation Percentage      object
Official Money                       object
Overall Putting Average              object
Percentage of Available Purse Won    object
Sand Save Percentage                 object
Scoring Average (Actual)             object
Scrambling                           object
dtype: object


In [13]:
df.head()

statistic,player_name,date,tournament,Driving Accuracy Percentage,Driving Distance,Greens in Regulation Percentage,Official Money,Overall Putting Average,Percentage of Available Purse Won,Sand Save Percentage,Scoring Average (Actual),Scrambling
0,A.J. Duncan,1988-04-03,KMart Greater Greensboro Open,58.93,283.5,58.33,"$1,810",NaN,.18,33.33,73.75,NaN
1,A.J. Duncan,1990-03-11,Honda Classic,44.64,264.1,52.78,"$1,810",NaN,.18,14.29,77.00,NaN
2,A.J. McInerney,2017-11-05,Shriners Hospitals for Children Open,50.00,317.8,76.39,NaN,1.694,2.22,22.22,69.50,52.94
3,A.J. McInerney,2018-06-10,FedEx St. Jude Classic,50.00,298.5,56.94,NaN,1.514,.22,33.33,70.75,61.29
4,Aaron Baddeley,2000-03-12,Honda Classic,64.29,283.6,70.83,NaN,1.625,NaN,50.00,70.25,66.67


In [14]:
#Cleaning the "Official Money" column so we can analyze it
df['Official Money'] = df['Official Money'].str.replace('\$', '', regex = True)
df['Official Money'] = df['Official Money'].str.replace(',', '', regex = True)
df['Official Money'] = df['Official Money'].str.replace('O', '0', regex = True)
df['Official Money'] = df['Official Money'].str.replace(' ', '', regex = True)
df['Official Money'] = df['Official Money'].str.strip()
df['Official Money'].replace("", np.nan, inplace=True)

In [15]:
#Converting data types from objects to float
df['Greens in Regulation Percentage']=df['Greens in Regulation Percentage'].astype('float')
df['Driving Accuracy Percentage']=df['Driving Accuracy Percentage'].astype('float')
df['Scrambling']=df['Scrambling'].astype('float')
df['Scoring Average (Actual)']=df['Scoring Average (Actual)'].astype('float')
df['Official Money']=df['Official Money'].astype('float')
df['Percentage of Available Purse Won']=df['Percentage of Available Purse Won'].astype('float')
df['Driving Distance']=df['Driving Distance'].astype('float')
df['Sand Save Percentage']=df['Sand Save Percentage'].astype('float')
df['Overall Putting Average']=df['Overall Putting Average'].astype('float')

In [16]:
print(df.dtypes)

statistic
player_name                           object
date                                  object
tournament                            object
Driving Accuracy Percentage          float64
Driving Distance                     float64
Greens in Regulation Percentage      float64
Official Money                       float64
Overall Putting Average              float64
Percentage of Available Purse Won    float64
Sand Save Percentage                 float64
Scoring Average (Actual)             float64
Scrambling                           float64
dtype: object


In [17]:
#Cleaning up the DataFrame - renaming certain columns
df.rename(columns = {'Greens in Regulation Percentage':'GIR',
                     'Driving Accuracy Percentage':'Fairways',
                     'Scrambling':'Scrambling',
                     'Scoring Average (Actual)':'Score',
                     'Official Money':'Money',
                     'Percentage of Available Purse Won':'Purse',
                     'Driving Distance':'Distance',
                     'Sand Save Percentage':'Sandies',
                     'Overall Putting Average':'Putts'}, inplace=True)

In [18]:
df.head()

statistic,player_name,date,tournament,Fairways,Distance,GIR,Money,Putts,Purse,Sandies,Score,Scrambling
0,A.J. Duncan,1988-04-03,KMart Greater Greensboro Open,58.93,283.5,58.33,1810.0,NaN,0.18,33.33,73.75,NaN
1,A.J. Duncan,1990-03-11,Honda Classic,44.64,264.1,52.78,1810.0,NaN,0.18,14.29,77.00,NaN
2,A.J. McInerney,2017-11-05,Shriners Hospitals for Children Open,50.00,317.8,76.39,NaN,1.694,2.22,22.22,69.50,52.94
3,A.J. McInerney,2018-06-10,FedEx St. Jude Classic,50.00,298.5,56.94,NaN,1.514,0.22,33.33,70.75,61.29
4,Aaron Baddeley,2000-03-12,Honda Classic,64.29,283.6,70.83,NaN,1.625,NaN,50.00,70.25,66.67


In [19]:
df.describe()

statistic,Fairways,Distance,GIR,Money,Putts,Purse,Sandies,Score,Scrambling
count,124669.000000,123306.000000,124543.000000,1.234740e+05,87568.000000,118148.000000,123011.000000,126125.000000,87574.000000
mean,65.607781,275.949496,66.555930,4.870611e+04,1.604443,1.407430,51.691787,70.651087,61.097847
std,10.898736,19.243917,8.459017,1.231114e+05,0.084746,2.585814,24.156935,1.862798,11.346915
min,17.860000,182.600000,20.830000,9.600000e+01,0.611000,0.130000,0.000000,63.250000,4.170000
25%,58.930000,261.500000,61.110000,4.265000e+03,1.556000,0.230000,36.360000,69.250000,53.570000
50%,66.070000,275.300000,66.670000,1.332500e+04,1.611000,0.520000,50.000000,70.500000,61.290000
75%,73.210000,289.800000,72.220000,3.987500e+04,1.653000,1.450000,66.670000,71.750000,68.750000
max,100.000000,362.100000,487.500000,2.250000e+06,2.056000,30.000000,200.000000,81.250000,100.000000


### Fixing errors in the data
There is at least one GIR value that is above 100% (in the chart above the max = 487.5) and one Sandies value that is above 100% (in the chart above the max = 200.0).  Will drop any rows where GIR or Sandies is above 100%:

In [20]:
df=df[df.GIR <= 100]
df=df[df.Sandies <= 100]

In [21]:
df.describe()

statistic,Fairways,Distance,GIR,Money,Putts,Purse,Sandies,Score,Scrambling
count,122818.000000,121486.000000,122848.000000,1.184600e+05,86385.000000,113663.000000,122848.000000,122691.000000,86391.000000
mean,65.584648,276.036268,66.498876,4.843005e+04,1.604132,1.391657,51.688042,70.651976,61.093638
std,10.904967,19.236936,8.347419,1.213998e+05,0.084406,2.558169,24.150509,1.852941,11.312598
min,17.860000,182.600000,20.830000,2.800000e+02,0.611000,0.130000,0.000000,63.250000,4.170000
25%,58.930000,261.500000,61.110000,4.332000e+03,1.556000,0.230000,36.360000,69.330000,53.570000
50%,66.070000,275.400000,66.670000,1.339200e+04,1.611000,0.520000,50.000000,70.500000,61.290000
75%,73.210000,289.900000,72.220000,3.990000e+04,1.653000,1.450000,66.670000,71.750000,68.750000
max,100.000000,362.100000,100.000000,2.250000e+06,2.056000,30.000000,100.000000,81.250000,100.000000


In [23]:
#Adding the year as a column to the dataframe
df['year'] = pd.DatetimeIndex(df['date']).year
df['month'] = pd.DatetimeIndex(df['date']).month
df.head()

statistic,player_name,date,tournament,Fairways,Distance,GIR,Money,Putts,Purse,Sandies,Score,Scrambling,year,month
0,A.J. Duncan,1988-04-03,KMart Greater Greensboro Open,58.93,283.5,58.33,1810.0,NaN,0.18,33.33,73.75,NaN,1988,4
1,A.J. Duncan,1990-03-11,Honda Classic,44.64,264.1,52.78,1810.0,NaN,0.18,14.29,77.00,NaN,1990,3
2,A.J. McInerney,2017-11-05,Shriners Hospitals for Children Open,50.00,317.8,76.39,NaN,1.694,2.22,22.22,69.50,52.94,2017,11
3,A.J. McInerney,2018-06-10,FedEx St. Jude Classic,50.00,298.5,56.94,NaN,1.514,0.22,33.33,70.75,61.29,2018,6
4,Aaron Baddeley,2000-03-12,Honda Classic,64.29,283.6,70.83,NaN,1.625,NaN,50.00,70.25,66.67,2000,3


# Predicting the 2019 Masters winner

## Analyzing prior Masters Tournament winners

In [29]:
#How the 2019 winner performed in 2018
df_2019winner = df.loc[(df['player_name'] == 'Tiger Woods') & (df['year']==2018)]
df_2019winner.head(50)

statistic,player_name,date,tournament,Fairways,Distance,GIR,Money,Putts,Purse,Sandies,Score,Scrambling,year,month
117776,Tiger Woods,2018-01-28,Farmers Insurance Open,30.36,302.5,58.33,59685.0,1.528,0.87,40.00,71.25,63.33,2018,1
117777,Tiger Woods,2018-02-25,The Honda Classic,58.93,318.9,66.67,151800.0,1.597,2.30,50.00,70.00,66.67,2018,2
117778,Tiger Woods,2018-03-11,Valspar Championship,59.62,296.3,66.67,572000.0,1.542,8.80,42.86,68.75,70.83,2018,3
117779,Tiger Woods,2018-03-18,Arnold Palmer Invitational presented by Master...,60.71,306.8,62.50,338200.0,1.486,3.80,55.56,69.50,70.37,2018,3
117780,Tiger Woods,2018-03-18,Corales Puntacana Resort & Club Championship,60.71,306.8,62.50,338200.0,1.486,3.80,55.56,69.50,70.37,2018,3
117781,Tiger Woods,2018-04-08,Masters Tournament,53.57,293.8,66.67,63663.0,1.625,0.58,33.33,72.25,50.00,2018,4
117782,Tiger Woods,2018-05-06,Wells Fargo Championship,44.64,330.3,70.83,17479.0,1.750,0.23,14.29,71.50,71.43,2018,5
117783,Tiger Woods,2018-05-13,THE PLAYERS Championship,62.50,297.3,69.44,225500.0,1.542,2.05,60.00,69.25,63.64,2018,5
117784,Tiger Woods,2018-06-03,the Memorial Tournament presented by Nationwide,71.43,300.9,75.00,76985.0,1.639,0.87,75.00,69.75,66.67,2018,6
117785,Tiger Woods,2018-07-01,Quicken Loans National,57.14,305.9,73.61,312400.0,1.514,4.40,50.00,67.25,57.89,2018,7


In [30]:
#How the 2018 winner performed in 2017
df_2018winner = df.loc[(df['player_name'] == 'Patrick Reed') & (df['year']==2017)]
df_2018winner.head(50)

statistic,player_name,date,tournament,Fairways,Distance,GIR,Money,Putts,Purse,Sandies,Score,Scrambling,year,month
94161,Patrick Reed,2017-01-08,SBS Tournament of Champions,81.67,279.3,83.33,210000.0,1.639,3.44,66.67,69.25,41.67,2017,1
94162,Patrick Reed,2017-01-22,CareerBuilder Challenge,48.21,291.3,59.72,113680.0,1.417,1.96,60.00,68.75,82.76,2017,1
94163,Patrick Reed,2017-02-05,Waste Management Phoenix Open,44.64,308.1,61.11,13534.0,1.611,0.20,42.86,71.25,60.71,2017,2
94164,Patrick Reed,2017-02-12,AT&T Pebble Beach Pro-Am,76.36,283.1,70.83,56520.0,1.625,0.79,42.86,70.25,42.86,2017,2
94165,Patrick Reed,2017-02-19,Genesis Open,50.00,282.5,55.56,15610.0,1.542,0.22,44.44,71.00,65.63,2017,2
94166,Patrick Reed,2017-03-05,World Golf Championships-Mexico Championship,60.71,309.6,59.72,47250.0,1.611,0.48,28.57,72.50,37.93,2017,3
94167,Patrick Reed,2017-03-12,Valspar Championship,61.54,291.6,59.72,28350.0,1.556,0.45,70.00,70.75,68.97,2017,3
94169,Patrick Reed,2017-05-07,Wells Fargo Championship,42.86,282.8,59.72,142500.0,1.528,1.90,33.33,70.75,72.41,2017,5
94170,Patrick Reed,2017-05-14,THE PLAYERS Championship,67.86,290.1,68.06,92138.0,1.708,0.88,45.45,72.00,39.13,2017,5
94171,Patrick Reed,2017-05-21,AT&T Byron Nelson,42.86,287.8,52.78,78214.0,1.389,1.04,77.78,69.00,73.53,2017,5


## Analyzing prior Masters Tournament top finishers

In [ ]:
#2016
#df_2016_winners = df.loc[(df['tournament'] == 'Masters Tournament') & (df['year']==2016)]
#df_2016_winners.nsmallest(10, 'Score')

In [ ]:
#2017
#Sergio Garcia won in the playoff
df_2017_winners = df.loc[(df['tournament'] == 'Masters Tournament') & (df['year']==2017)]
df_2017_winners.nsmallest(10, 'Score')

In [ ]:
#2018
df_2018_winners = df.loc[(df['tournament'] == 'Masters Tournament') & (df['year']==2018)]
df_2018_winners.nsmallest(10, 'Score')

In [43]:
#2019
df_2019_winners = df.loc[(df['tournament'] == 'Masters Tournament') & (df['year']==2019)]
df_2019_winners.nsmallest(10, 'Score')

statistic,player_name,date,tournament,Fairways,Distance,GIR,Money,Putts,Purse,Sandies,Score,Scrambling,year,month
117798,Tiger Woods,2019-04-14,Masters Tournament,62.50,294.6,80.56,2070000.0,1.667,NaN,66.67,68.75,50.00,2019,4
17525,Brooks Koepka,2019-04-14,Masters Tournament,69.64,313.6,73.61,858667.0,1.639,NaN,100.00,69.00,68.42,2019,4
38616,Dustin Johnson,2019-04-14,Masters Tournament,60.71,308.0,70.83,858667.0,1.569,NaN,100.00,69.00,80.95,2019,4
127350,Xander Schauffele,2019-04-14,Masters Tournament,62.50,305.8,70.83,858667.0,1.597,NaN,57.14,69.00,57.14,2019,4
40810,Francesco Molinari,2019-04-14,Masters Tournament,73.21,294.8,65.28,403938.0,1.458,NaN,66.67,69.25,84.00,2019,4
54817,Jason Day,2019-04-14,Masters Tournament,66.07,296.5,70.83,403938.0,1.556,NaN,33.33,69.25,71.43,2019,4
123063,Tony Finau,2019-04-14,Masters Tournament,67.86,316.3,66.67,403938.0,1.556,NaN,63.64,69.25,79.17,2019,4
126086,Webb Simpson,2019-04-14,Masters Tournament,83.93,283.1,68.06,403938.0,1.556,NaN,50.00,69.25,78.26,2019,4
68306,Jon Rahm,2019-04-14,Masters Tournament,76.79,308.4,70.83,310500.0,1.694,NaN,71.43,69.50,80.95,2019,4
94045,Patrick Cantlay,2019-04-14,Masters Tournament,64.29,299.9,61.11,310500.0,1.472,NaN,71.43,69.50,67.86,2019,4


## Reducing the data down to the last 2 years prior to the 2019 Masters (2017-2018)

In [44]:
df_2019winner = df[(df.year > 2016) & (df.year < 2019)]
df_2019winner.head()

statistic,player_name,date,tournament,Fairways,Distance,GIR,Money,Putts,Purse,Sandies,Score,Scrambling,year,month
2,A.J. McInerney,2017-11-05,Shriners Hospitals for Children Open,50.00,317.8,76.39,NaN,1.694,2.22,22.22,69.50,52.94,2017,11
3,A.J. McInerney,2018-06-10,FedEx St. Jude Classic,50.00,298.5,56.94,NaN,1.514,0.22,33.33,70.75,61.29,2018,6
220,Aaron Baddeley,2017-01-08,SBS Tournament of Champions,76.67,287.9,81.94,70000.0,1.750,1.15,50.00,71.50,46.15,2017,1
221,Aaron Baddeley,2017-01-22,CareerBuilder Challenge,50.00,283.6,55.56,10962.0,1.486,0.19,57.14,71.75,65.63,2017,1
222,Aaron Baddeley,2017-03-19,Arnold Palmer Invitational presented by Master...,53.57,285.9,58.33,33060.0,1.583,0.38,55.56,72.25,63.33,2017,3


## Creating a "Trending" score

## Aggregating the data by player

In [46]:
aggregation_functions = {'Score': 'mean', 'Fairways': 'mean', 'Distance': 'mean', 'GIR': 'mean', 'Putts': 'mean', 'Sandies': 'mean', 'Scrambling': 'mean', 'Money': 'sum'} #'player_name': 'first'
df_2019winner_agg = df.groupby(df_2019winner['player_name']).aggregate(aggregation_functions)
df_2019winner_agg.nlargest(30, 'Money')
#df_2019winner_agg.head()

,Score,Fairways,Distance,GIR,Putts,Sandies,Scrambling,Money
player_name,,,,,,,,
Justin Thomas,68.925676,58.864324,311.280645,69.482162,1.575054,55.189189,63.066757,18319086.0
Dustin Johnson,68.678571,58.024857,314.150000,70.754286,1.586171,50.920857,64.209143,16103128.0
Brooks Koepka,69.570312,57.875000,311.696552,65.885625,1.581625,49.109063,61.052187,14132752.0
Marc Leishman,69.672222,60.407556,299.272500,68.270444,1.605600,49.296667,61.625111,13339364.0
Bryson DeChambeau,69.263889,63.123333,305.311111,69.598889,1.606361,54.578611,61.037500,12871237.0
Justin Rose,69.340909,63.103939,301.712500,68.308182,1.591818,61.073030,63.352121,12039631.0
Jordan Spieth,68.878571,60.152857,298.105714,70.595714,1.587686,54.055143,62.823429,11761628.0
Rickie Fowler,69.283784,65.844054,300.402703,68.393514,1.585568,62.194324,64.517838,10710484.0
Xander Schauffele,69.945946,60.922703,308.430000,67.228649,1.612162,55.596486,60.195405,9697539.0


## Selecting independent variables

### Remove variables with high P-values

#### Comparing the variables to "Score" (average number of strokes per completed round)
None have P-values above 0.05

In [ ]:
#Score_model = sm.OLS.from_formula("Score ~ Fairways + Distance + GIR + Putts + Sandies + Scrambling", data=df)
#Score_results = Score_model.fit()
#print("P-values:"), print(Score_results.pvalues), print("\nR-squared:"), print(Score_results.rsquared)
#print(Score_results.summary())

#Note:
#P-value is the probability of getting a sample proportion at least this extreme.  Ranges from zero to 100%. 
#NThe P>|t| value in the OLS summary reflects a two-sided P-value, and needs to be divided by 2 to reflect the single sided value.
#P-values quoted in the summary of this notebook reflect the single sided P-value

In [33]:
#Fairways
Score_Fairways_model = sm.OLS.from_formula("Score ~ Fairways", data=df_2019winner)
Score_Fairways_results = Score_Fairways_model.fit()
print("Score + Fairways | P-value:", Score_Fairways_results.pvalues[1]/2, "| R-squared:", Score_Fairways_results.rsquared)

#Distance
Score_Distance_model = sm.OLS.from_formula("Score ~ Distance", data=df_2019winner)
Score_Distance_results = Score_Distance_model.fit()
print("Score + Distance | P-value:", Score_Distance_results.pvalues[1]/2, "| R-squared:", Score_Distance_results.rsquared)

#GIR
Score_GIR_model = sm.OLS.from_formula("Score ~ GIR", data=df_2019winner)
Score_GIR_results = Score_GIR_model.fit()
print("Score + GIR | P-value:", Score_GIR_results.pvalues[1]/2, "| R-squared:", Score_GIR_results.rsquared)

#Putts
Score_Putts_model = sm.OLS.from_formula("Score ~ Putts", data=df_2019winner)
Score_Putts_results = Score_Putts_model.fit()
print("Score + Putts | P-value:", Score_Putts_results.pvalues[1]/2, "| R-squared:", Score_Putts_results.rsquared)

#Sandies
Score_Sandies_model = sm.OLS.from_formula("Score ~ Sandies", data=df_2019winner)
Score_Sandies_results = Score_Sandies_model.fit()
print("Score + Sandies | P-value:", Score_Sandies_results.pvalues[1]/2, "| R-squared:", Score_Sandies_results.rsquared)

#Scrambling
Score_Scrambling_model = sm.OLS.from_formula("Score ~ Scrambling", data=df_2019winner)
Score_Scrambling_results = Score_Scrambling_model.fit()
print("Score + Scrambling | P-value:", Score_Scrambling_results.pvalues[1]/2, "| R-squared:", Score_Scrambling_results.rsquared)

#Money
Score_Money_model = sm.OLS.from_formula("Score ~ Money", data=df_2019winner)
Score_Money_results = Score_Money_model.fit()
print("Score + Money | P-value:", Score_Money_results.pvalues[1]/2, "| R-squared:", Score_Money_results.rsquared)


#Note:
#P-value is the probability of getting a sample proportion at least this extreme.  Ranges from zero to 100%. 
#The P>|t| value in the OLS summary reflects a two-sided P-value, and needs to be divided by 2 to reflect the single sided value.
#P-values quoted in the summary of this notebook reflect the single sided P-value

Score + Fairways | P-value: 5.1802895351830215e-58 | R-squared: 0.026161667829114
Score + Distance | P-value: 1.0073211812813001e-80 | R-squared: 0.039712129770909765
Score + GIR | P-value: 0.0 | R-squared: 0.3248347882982755
Score + Putts | P-value: 1.3681100441629318e-188 | R-squared: 0.08474755589044958
Score + Sandies | P-value: 3.344372734555088e-83 | R-squared: 0.0377345708982918
Score + Scrambling | P-value: 0.0 | R-squared: 0.20602232664351627
Score + Money | P-value: 0.0 | R-squared: 0.16060020384668805


In [37]:
#Fairways
Money_Fairways_model = sm.OLS.from_formula("Money ~ Fairways", data=df_2019winner)
Money_Fairways_results = Money_Fairways_model.fit()
print("Money + Fairways | P-value:", Money_Fairways_results.pvalues[1]/2, "| R-squared:", Money_Fairways_results.rsquared)

#Distance
Money_Distance_model = sm.OLS.from_formula("Money ~ Distance", data=df_2019winner)
Money_Distance_results = Money_Distance_model.fit()
print("Money + Distance | P-value:", Money_Distance_results.pvalues[1]/2, "| R-squared:", Money_Distance_results.rsquared)

#GIR
Money_GIR_model = sm.OLS.from_formula("Money ~ GIR", data=df_2019winner)
Money_GIR_results = Money_GIR_model.fit()
print("Money + GIR | P-value:", Money_GIR_results.pvalues[1]/2, "| R-squared:", Money_GIR_results.rsquared)

#Putts
Money_Putts_model = sm.OLS.from_formula("Money ~ Putts", data=df_2019winner)
Money_Putts_results = Money_Putts_model.fit()
print("Money + Putts | P-value:", Money_Putts_results.pvalues[1]/2, "| R-squared:", Money_Putts_results.rsquared)

#Sandies
Money_Sandies_model = sm.OLS.from_formula("Money ~ Sandies", data=df_2019winner)
Money_Sandies_results = Money_Sandies_model.fit()
print("Money + Sandies | P-value:", Money_Sandies_results.pvalues[1]/2, "| R-squared:", Money_Sandies_results.rsquared)

#Scrambling
Money_Scrambling_model = sm.OLS.from_formula("Money ~ Scrambling", data=df_2019winner)
Money_Scrambling_results = Money_Scrambling_model.fit()
print("Money + Scrambling | P-value:", Money_Scrambling_results.pvalues[1]/2, "| R-squared:", Money_Scrambling_results.rsquared)

#Money
Money_Score_model = sm.OLS.from_formula("Money ~ Score", data=df_2019winner)
Money_Score_results = Money_Score_model.fit()
print("Money + Score | P-value:", Money_Score_results.pvalues[1]/2, "| R-squared:", Money_Score_results.rsquared)


Money + Fairways | P-value: 2.56619087601171e-12 | R-squared: 0.005362370231914881
Money + Distance | P-value: 4.2186344566794755e-38 | R-squared: 0.019816852425552756
Money + GIR | P-value: 1.1526328880433418e-85 | R-squared: 0.042348581822712505
Money + Putts | P-value: 4.413602904856733e-75 | R-squared: 0.03708058494144206
Money + Sandies | P-value: 5.346139871034969e-23 | R-squared: 0.010793203643401639
Money + Scrambling | P-value: 2.491462117911033e-103 | R-squared: 0.05108139011932089
Money + Score | P-value: 0.0 | R-squared: 0.16060020384668838


### Comparing correlation between variables

In [34]:
df_2019winner.corr()

statistic,Fairways,Distance,GIR,Money,Putts,Purse,Sandies,Score,Scrambling,year,month
statistic,,,,,,,,,,,
Fairways,1.000000,-0.302507,0.388188,0.073228,0.198287,0.073323,-0.006810,-0.161746,0.060299,0.083470,0.106202
Distance,-0.302507,1.000000,0.137533,0.140772,0.079965,0.121717,-0.025984,-0.199279,-0.059969,0.162914,0.147300
GIR,0.388188,0.137533,1.000000,0.205788,0.466389,0.232361,0.007647,-0.569943,0.035502,0.089549,0.226333
Money,0.073228,0.140772,0.205788,1.000000,-0.192563,0.972651,0.103890,-0.400750,0.226012,0.015669,0.019462
Putts,0.198287,0.079965,0.466389,-0.192563,1.000000,-0.217292,-0.205633,0.291114,-0.433270,0.010414,0.165429
Purse,0.073323,0.121717,0.232361,0.972651,-0.217292,1.000000,0.117810,-0.437808,0.259735,-0.002965,-0.012063
Sandies,-0.006810,-0.025984,0.007647,0.103890,-0.205633,0.117810,1.000000,-0.194254,0.317009,-0.004904,0.024083
Score,-0.161746,-0.199279,-0.569943,-0.400750,0.291114,-0.437808,-0.194254,1.000000,-0.453897,-0.109199,-0.107400
Scrambling,0.060299,-0.059969,0.035502,0.226012,-0.433270,0.259735,0.317009,-0.453897,1.000000,0.012386,-0.099994


## Creating the model

In [38]:
Model = sm.OLS.from_formula("Money ~ GIR + Putts + Scrambling", data=df_2019winner)
Results = Model.fit()
print(Results.summary())

                            OLS Regression Results                            
Dep. Variable:                  Money   R-squared:                       0.162
Model:                            OLS   Adj. R-squared:                  0.161
Method:                 Least Squares   F-statistic:                     568.9
Date:                Sat, 05 Sep 2020   Prob (F-statistic):               0.00
Time:                        09:09:09   Log-Likelihood:            -1.2016e+05
No. Observations:                8862   AIC:                         2.403e+05
Df Residuals:                    8858   BIC:                         2.404e+05
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   8.203e+05   5.27e+04     15.569      0.0

In [39]:
df_2019winner['Prediction'] = 820300 + (9656.4244*df_2019winner.GIR) + (-.00009014*df_2019winner.Putts) + (1398.0181*df_2019winner.Scrambling)
df_2019winner.nsmallest(10, 'Prediction')

/Users/nicole/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


statistic,player_name,date,tournament,Fairways,Distance,GIR,Money,Putts,Purse,Sandies,Score,Scrambling,year,month,Prediction
115242,Steven Bowditch,2016-07-03,Barbasol Championship,26.79,299.0,26.39,52000.0,1.486,0.55,50.00,77.00,47.17,2016,7,1.141078e+06
115243,Steven Bowditch,2016-07-03,World Golf Championships-Bridgestone Invitational,26.79,299.0,26.39,52000.0,1.486,0.55,50.00,77.00,47.17,2016,7,1.141078e+06
12166,Bowen Xiao,2018-10-28,World Golf Championships-HSBC Champions,60.71,NaN,38.89,NaN,1.583,0.44,66.67,79.25,31.82,2018,10,1.240323e+06
15238,Brett Rumford,2018-10-28,World Golf Championships-HSBC Champions,41.07,NaN,37.50,NaN,1.486,0.45,60.00,77.25,42.22,2018,10,1.241440e+06
93015,Pablo Larrazabal,2017-03-05,World Golf Championships-Mexico Championship,41.07,290.6,38.89,NaN,1.458,0.45,0.00,74.75,56.82,2017,3,1.275274e+06
24969,Chris Wood,2017-08-06,World Golf Championships-Bridgestone Invitational,46.43,287.4,40.28,NaN,1.583,0.45,50.00,74.25,48.84,2017,8,1.277540e+06
53845,Jaco Ahlers,2018-08-05,World Golf Championships-Bridgestone Invitational,30.36,311.8,41.67,NaN,1.639,0.47,25.00,75.75,40.48,2018,8,1.279275e+06
47299,Greg Chalmers,2016-07-17,The Open Championship,51.79,282.1,40.28,18331.0,1.542,0.20,25.00,76.25,51.16,2016,7,1.280783e+06
4354,Ben Crane,2018-02-25,The Honda Classic,55.36,270.8,40.28,12606.0,1.528,0.19,55.56,74.00,51.16,2018,2,1.280783e+06
30350,Danny Willett,2017-08-06,World Golf Championships-Bridgestone Invitational,32.14,281.1,41.67,43500.0,1.597,0.45,62.50,75.25,45.24,2017,8,1.285930e+06
